RNN on DCDC

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function, division
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

import matplotlib.pyplot as plt
# plot param
%matplotlib inline
plt.rcParams['figure.figsize'] = (22.0, 12.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

First, manipulate the input and output data

Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [2]:
# parse the controller file
f = open('controller.scs', "r")

In [3]:
lines = []

Skip the header part and go straight to the state-action pair

In [4]:
for line in f:
    if '#MATRIX:DATA\n' in line:                
        for line in f: # now you are at the lines you want
            # skip the #BEGIN 
            # read the state-actions
            lines = f.readlines()

In [5]:
# take the state as the train dataset
ltrain_dataset = []
for x in lines:
    ltrain_dataset.append(x.split(' ')[0])
del ltrain_dataset[-1] # delete the string #END at the end of the file 

In [6]:
# convert to the numpy array with float32 data type
train_dataset = np.asarray(ltrain_dataset)
train_dataset = train_dataset.astype(np.float32)
train_dataset.shape

(593089,)

In [7]:
# take the action(s) [column 1:-1] / the rest of the integer except the state
ltrain_label = []
for x in lines:
    ltrain_label.append(x.split(' ')[1:-1])
del ltrain_label[-1] # delete the string #END at the end of the file  

In [8]:
# convert to numpy array, note that the result is still not in one hot encoding format
train_label = np.asarray(ltrain_label)
train_label

array([list(['1']), list(['1']), list(['1']), ..., list(['0']),
       list(['0']), list(['0'])], dtype=object)

Iterate from here

In [9]:
# define number of samples
num_samples = train_dataset.shape[0]
num_samples

593089

In [10]:
# create now array to be filled by the encoded label
train_label_hot = np.zeros([num_samples,2], dtype=np.float32)
train_label_hot.shape

(593089, 2)

In [11]:
ltrain_label[0]

['1']

In [12]:
# encode label to one hot encoding format
for i in range(num_samples):
    if train_label[i] == ['0']:
        train_label_hot[i] = [1, 0]
    elif train_label[i] == ['1']:
        train_label_hot[i] = [0, 1]
    elif train_label[i] == ['0','1']:
        train_label_hot[i] = [1, 1]

Small dataset

In [13]:
# define the new number of samples for debugging purpose
num_samples = 40
# define batch offset to limit the desired dataset index
batch_off = 20000+16800-87-25
# slice the train label to the desired index
train_label_hot = train_label_hot[batch_off:batch_off + num_samples]
train_label_hot.shape

(40, 2)

In [14]:
# do the same thing to the dataset and add dimension on the column
train_dataset_reform = train_dataset[batch_off:batch_off + num_samples, None]
train_dataset_reform.shape

(40, 1)

Execute this to get the full batch

In [13]:
train_dataset_reform = train_dataset[:, None]
train_dataset_reform.shape

(593089, 1)

# NN

In [15]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [16]:
train_dataset_reform[0]

array([62388.], dtype=float32)

In [17]:
train_dataset_reform[-1]

array([62576.], dtype=float32)

In [18]:
train_label_hot[0]

array([0., 1.], dtype=float32)

In [19]:
train_label_hot[-1]

array([1., 1.], dtype=float32)

In [20]:
num_samples

40

RNN example

In [20]:
total_series_length = num_samples
truncated_backprop_length = 10
state_size = 1
num_classes = 2
echo_step = 2
batch_size = 7
num_batches = total_series_length//batch_size//truncated_backprop_length

In [21]:
num_samples/7/15

5648.466666666666

In [22]:
num_batches

5648

In [23]:
# computational graph

In [24]:
# 5 x 15
batchX_placeholder = tf.placeholder(tf.float32, [batch_size, truncated_backprop_length])
# 5 x 15
batchY_placeholder = tf.placeholder(tf.float32, [batch_size, truncated_backprop_length, num_classes])

# initial state 5 x 4
init_state = tf.placeholder(tf.float32, [batch_size, state_size])

In [25]:
init_state

<tf.Tensor 'Placeholder_2:0' shape=(7, 1) dtype=float32>

In [26]:
# weight and bias

In [27]:
# W size 5 x 4
W = tf.Variable(np.random.rand(state_size+1, state_size), dtype=tf.float32)
# b size 1 x 4
b = tf.Variable(np.zeros((1,state_size)), dtype=tf.float32)

# 4 x 2
W2 = tf.Variable(np.random.rand(state_size, num_classes),dtype=tf.float32)
# 1 x 2
b2 = tf.Variable(np.zeros((1,num_classes)), dtype=tf.float32)

In [28]:
# unpack
# Unpack columns/slice each column (5 x 15) -> (5x1) 15  
inputs_series = tf.unstack(batchX_placeholder, axis=1)
labels_series = tf.unstack(batchY_placeholder, axis=1)

In [29]:
inputs_series

[<tf.Tensor 'unstack:0' shape=(7,) dtype=float32>,
 <tf.Tensor 'unstack:1' shape=(7,) dtype=float32>,
 <tf.Tensor 'unstack:2' shape=(7,) dtype=float32>,
 <tf.Tensor 'unstack:3' shape=(7,) dtype=float32>,
 <tf.Tensor 'unstack:4' shape=(7,) dtype=float32>,
 <tf.Tensor 'unstack:5' shape=(7,) dtype=float32>,
 <tf.Tensor 'unstack:6' shape=(7,) dtype=float32>,
 <tf.Tensor 'unstack:7' shape=(7,) dtype=float32>,
 <tf.Tensor 'unstack:8' shape=(7,) dtype=float32>,
 <tf.Tensor 'unstack:9' shape=(7,) dtype=float32>,
 <tf.Tensor 'unstack:10' shape=(7,) dtype=float32>,
 <tf.Tensor 'unstack:11' shape=(7,) dtype=float32>,
 <tf.Tensor 'unstack:12' shape=(7,) dtype=float32>,
 <tf.Tensor 'unstack:13' shape=(7,) dtype=float32>,
 <tf.Tensor 'unstack:14' shape=(7,) dtype=float32>]

In [30]:
labels_series

[<tf.Tensor 'unstack_1:0' shape=(7, 2) dtype=float32>,
 <tf.Tensor 'unstack_1:1' shape=(7, 2) dtype=float32>,
 <tf.Tensor 'unstack_1:2' shape=(7, 2) dtype=float32>,
 <tf.Tensor 'unstack_1:3' shape=(7, 2) dtype=float32>,
 <tf.Tensor 'unstack_1:4' shape=(7, 2) dtype=float32>,
 <tf.Tensor 'unstack_1:5' shape=(7, 2) dtype=float32>,
 <tf.Tensor 'unstack_1:6' shape=(7, 2) dtype=float32>,
 <tf.Tensor 'unstack_1:7' shape=(7, 2) dtype=float32>,
 <tf.Tensor 'unstack_1:8' shape=(7, 2) dtype=float32>,
 <tf.Tensor 'unstack_1:9' shape=(7, 2) dtype=float32>,
 <tf.Tensor 'unstack_1:10' shape=(7, 2) dtype=float32>,
 <tf.Tensor 'unstack_1:11' shape=(7, 2) dtype=float32>,
 <tf.Tensor 'unstack_1:12' shape=(7, 2) dtype=float32>,
 <tf.Tensor 'unstack_1:13' shape=(7, 2) dtype=float32>,
 <tf.Tensor 'unstack_1:14' shape=(7, 2) dtype=float32>]

In [31]:
# Forward pass
current_state = init_state # 5x4
states_series = []
for current_input in inputs_series:
    # 5x1
    current_input = tf.reshape(current_input, [batch_size, 1]) 
    # 5x1 concat 5x4 -> 5x5 
    input_and_state_concatenated = tf.concat([current_input, current_state],1)  # Increasing number of columns
    # 5x5, 5x4 -> 5x4 + 1x4 = 5x4
    # next_state = tf.tanh(tf.matmul(input_and_state_concatenated, W) + b)  # Broadcasted addition
    next_state = tf.nn.relu(tf.matmul(input_and_state_concatenated, W) + b)  # Broadcasted addition
    
    states_series.append(next_state)
    current_state = next_state

In [32]:
# loss calculation

In [33]:
# 5x4 * 4x2 + 1x2 = 5x2
logits_series = [tf.matmul(state, W2) + b2 for state in states_series] #Broadcasted addition

# the same size with the batch for the training
# predictions_series = [tf.nn.softmax(logits) for logits in logits_series]
predictions_series = [tf.round(tf.sigmoid(logits)) for logits in logits_series]

# compute the loss with cross entropy (it will be probabilistic 0-1 compare it with label)
# losses = [tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits,labels = labels) for logits, labels in zip(logits_series,labels_series)]
losses = [tf.nn.sigmoid_cross_entropy_with_logits(logits = logits,labels = labels) for logits, labels in zip(logits_series,labels_series)]

# make the loss scalar
total_loss = tf.reduce_mean(losses)
# gradient step
train_step = tf.train.AdagradOptimizer(0.3).minimize(total_loss)

In [34]:
train_label_hot.shape

(593089, 2)

In [35]:
train_dataset_reform.shape

(593089, 1)

In [36]:
x = train_dataset_reform.reshape((batch_size,-1))
y = train_label_hot.reshape((batch_size, -1, num_classes))

In [37]:
x.shape

(7, 84727)

In [38]:
y.shape

(7, 84727, 2)

In [39]:
num_batches

5648

In [40]:
5648*7*15

593040

In [41]:
num_epochs = 2

In [42]:
num_batches*15*7

593040

In [43]:
def accuracy_independent(predictions, labels):
  return (100.0 * np.mean(np.asarray(predictions) == np.asarray(labels)))

In [47]:
with tf.Session() as sess:
    # sess.run(tf.initialize_all_variables())
    sess.run(tf.global_variables_initializer())
    # plt.ion()
    # plt.figure()
    # plt.show()
    loss_list = []
    
    
    for epoch_idx in range(1): # 100
        _current_state = np.zeros((batch_size, state_size))

        print("New data, epoch", epoch_idx)

        for batch_idx in range(1): # 666
            start_idx = batch_idx * truncated_backprop_length
            end_idx = start_idx + truncated_backprop_length
            # all 5 rows, certain numbers of column which the total = 15 columns
            batchX = x[:,start_idx:end_idx]
            batchY = y[:,start_idx:end_idx]
            
            # calculate loss, gradient step, 
            _total_loss, _train_step, _current_state, _predictions_series, _labels_series = sess.run(
                [total_loss, train_step, current_state, predictions_series, labels_series],
                feed_dict={
                    batchX_placeholder:batchX,
                    batchY_placeholder:batchY,
                    # it was part of computational graph, where the init_state is the placeholder
                    init_state:_current_state
                })

            loss_list.append(_total_loss)
            # print(_predictions_series)
            # print(_labels_series)
           
            if (batch_idx%(num_batches//10) == 0):
                print("Step",batch_idx, "Loss", _total_loss)
                # plot(loss_list, _predictions_series, batchX, batchY)
"""
    # accuracy
    _current_state = np.zeros((batch_size, state_size))
    acc = 0
    total_acc = 0
    for batch_idx in range(num_batches):
        start_idx = batch_idx * truncated_backprop_length
        end_idx = start_idx + truncated_backprop_length
        
        batchX = x[:,start_idx:end_idx]
        batchY = y[:,start_idx:end_idx]

        # calculate loss, gradient step, 
        _predictions_series, _labels_series = sess.run([predictions_series, labels_series],
            feed_dict={
                batchX_placeholder:batchX,
                batchY_placeholder:batchY,
                # it was part of computational graph, where the init_state is the placeholder
                init_state:_current_state
            })
        acc = accuracy_independent( _predictions_series, _labels_series)
        print("batch:",batch_idx,", accuracy:",acc)
        total_acc += acc
    
    print(total_acc/num_batches)
# plt.ioff()
# plt.show()
"""

New data, epoch 0
Step 0 Loss 6300.058


'\n    # accuracy\n    _current_state = np.zeros((batch_size, state_size))\n    acc = 0\n    total_acc = 0\n    for batch_idx in range(num_batches):\n        start_idx = batch_idx * truncated_backprop_length\n        end_idx = start_idx + truncated_backprop_length\n        \n        batchX = x[:,start_idx:end_idx]\n        batchY = y[:,start_idx:end_idx]\n\n        # calculate loss, gradient step, \n        _predictions_series, _labels_series = sess.run([predictions_series, labels_series],\n            feed_dict={\n                batchX_placeholder:batchX,\n                batchY_placeholder:batchY,\n                # it was part of computational graph, where the init_state is the placeholder\n                init_state:_current_state\n            })\n        acc = accuracy_independent( _predictions_series, _labels_series)\n        print("batch:",batch_idx,", accuracy:",acc)\n        total_acc += acc\n    \n    print(total_acc/num_batches)\n# plt.ioff()\n# plt.show()\n'

In [89]:
with tf.Session() as sess:
    # sess.run(tf.initialize_all_variables())
    sess.run(tf.global_variables_initializer())
    # plt.ion()
    # plt.figure()
    # plt.show()
    loss_list = []
    
    print("TRAINING")
    for epoch_idx in range(10): # 100
        _current_state = np.zeros((batch_size, state_size))

        print("New data, epoch", epoch_idx)

        for batch_idx in range(num_batches): # 666
            start_idx = batch_idx * truncated_backprop_length
            end_idx = start_idx + truncated_backprop_length
            # all 5 rows, certain numbers of column which the total = 15 columns
            batchX = x[:,start_idx:end_idx]
            batchY = y[:,start_idx:end_idx]
            
            # calculate loss, gradient step, 
            _total_loss, _train_step, _current_state, _predictions_series, _labels_series = sess.run(
                [total_loss, train_step, current_state, predictions_series, labels_series],
                feed_dict={
                    batchX_placeholder:batchX,
                    batchY_placeholder:batchY,
                    # it was part of computational graph, where the init_state is the placeholder
                    init_state:_current_state
                })

            loss_list.append(_total_loss)
            # print(_predictions_series)
            # print(_labels_series)
           
            if (batch_idx%(num_batches//10) == 0):
                print("Step",batch_idx, "Loss", _total_loss)
                # plot(loss_list, _predictions_series, batchX, batchY)

    # accuracy
    print("TESTING")
    _current_state = np.zeros((batch_size, state_size))
    acc = 0
    total_acc = 0
    for batch_idx in range(num_batches):
        start_idx = batch_idx * truncated_backprop_length
        end_idx = start_idx + truncated_backprop_length
        
        batchX = x[:,start_idx:end_idx]
        batchY = y[:,start_idx:end_idx]

        # calculate loss, gradient step, 
        _predictions_series, _labels_series = sess.run([predictions_series, labels_series],
            feed_dict={
                batchX_placeholder:batchX,
                batchY_placeholder:batchY,
                # it was part of computational graph, where the init_state is the placeholder
                init_state:_current_state
            })
        acc = accuracy_independent( _predictions_series, _labels_series)
        print("batch:",batch_idx,", accuracy:",acc)
        total_acc += acc
    
    print(total_acc/num_batches)
# plt.ioff()
# plt.show()

TRAINING
New data, epoch 0
Step 0 Loss 0.6145366
Step 564 Loss 0.39018336
Step 1128 Loss 0.40248197
Step 1692 Loss 0.35353068
Step 2256 Loss 0.6734549
Step 2820 Loss 0.48414764
Step 3384 Loss 0.6703673
Step 3948 Loss 0.50980175
Step 4512 Loss 0.26890025
Step 5076 Loss 0.6859591
Step 5640 Loss 0.39872596
New data, epoch 1
Step 0 Loss 0.5216932
Step 564 Loss 0.3729603
Step 1128 Loss 0.40798077
Step 1692 Loss 0.35940143
Step 2256 Loss 0.6738761
Step 2820 Loss 0.49470764
Step 3384 Loss 0.6700119
Step 3948 Loss 0.50538677
Step 4512 Loss 0.2694867
Step 5076 Loss 0.68852305
Step 5640 Loss 0.40143088
New data, epoch 2
Step 0 Loss 0.5087365
Step 564 Loss 0.36769417
Step 1128 Loss 0.41008374
Step 1692 Loss 0.3607785
Step 2256 Loss 0.6740317
Step 2820 Loss 0.5002517
Step 3384 Loss 0.6710399
Step 3948 Loss 0.5028255
Step 4512 Loss 0.26995072
Step 5076 Loss 0.68896353
Step 5640 Loss 0.40305042
New data, epoch 3
Step 0 Loss 0.5007702
Step 564 Loss 0.36376894
Step 1128 Loss 0.4125583
Step 1692 Loss 0

batch: 244 , accuracy: 92.85714285714286
batch: 245 , accuracy: 92.85714285714286
batch: 246 , accuracy: 92.85714285714286
batch: 247 , accuracy: 92.85714285714286
batch: 248 , accuracy: 92.85714285714286
batch: 249 , accuracy: 92.85714285714286
batch: 250 , accuracy: 92.85714285714286
batch: 251 , accuracy: 92.85714285714286
batch: 252 , accuracy: 92.85714285714286
batch: 253 , accuracy: 92.85714285714286
batch: 254 , accuracy: 92.85714285714286
batch: 255 , accuracy: 92.85714285714286
batch: 256 , accuracy: 92.85714285714286
batch: 257 , accuracy: 92.85714285714286
batch: 258 , accuracy: 92.85714285714286
batch: 259 , accuracy: 92.85714285714286
batch: 260 , accuracy: 90.0
batch: 261 , accuracy: 85.71428571428571
batch: 262 , accuracy: 84.28571428571429
batch: 263 , accuracy: 78.57142857142857
batch: 264 , accuracy: 78.57142857142857
batch: 265 , accuracy: 71.42857142857143
batch: 266 , accuracy: 71.42857142857143
batch: 267 , accuracy: 65.71428571428571
batch: 268 , accuracy: 64.285

batch: 537 , accuracy: 56.666666666666664
batch: 538 , accuracy: 50.0
batch: 539 , accuracy: 50.0
batch: 540 , accuracy: 50.0
batch: 541 , accuracy: 50.0
batch: 542 , accuracy: 50.0
batch: 543 , accuracy: 50.0
batch: 544 , accuracy: 50.0
batch: 545 , accuracy: 50.0
batch: 546 , accuracy: 50.0
batch: 547 , accuracy: 50.0
batch: 548 , accuracy: 50.0
batch: 549 , accuracy: 50.0
batch: 550 , accuracy: 50.0
batch: 551 , accuracy: 50.0
batch: 552 , accuracy: 54.285714285714285
batch: 553 , accuracy: 57.14285714285714
batch: 554 , accuracy: 61.904761904761905
batch: 555 , accuracy: 64.28571428571429
batch: 556 , accuracy: 69.04761904761905
batch: 557 , accuracy: 71.42857142857143
batch: 558 , accuracy: 76.66666666666667
batch: 559 , accuracy: 78.57142857142857
batch: 560 , accuracy: 83.80952380952381
batch: 561 , accuracy: 85.71428571428571
batch: 562 , accuracy: 91.42857142857143
batch: 563 , accuracy: 92.85714285714286
batch: 564 , accuracy: 92.85714285714286
batch: 565 , accuracy: 92.85714

batch: 827 , accuracy: 85.71428571428571
batch: 828 , accuracy: 89.04761904761904
batch: 829 , accuracy: 92.85714285714286
batch: 830 , accuracy: 92.85714285714286
batch: 831 , accuracy: 92.85714285714286
batch: 832 , accuracy: 92.85714285714286
batch: 833 , accuracy: 92.85714285714286
batch: 834 , accuracy: 92.85714285714286
batch: 835 , accuracy: 92.85714285714286
batch: 836 , accuracy: 92.85714285714286
batch: 837 , accuracy: 92.85714285714286
batch: 838 , accuracy: 92.85714285714286
batch: 839 , accuracy: 92.85714285714286
batch: 840 , accuracy: 92.85714285714286
batch: 841 , accuracy: 92.85714285714286
batch: 842 , accuracy: 92.85714285714286
batch: 843 , accuracy: 92.85714285714286
batch: 844 , accuracy: 92.85714285714286
batch: 845 , accuracy: 92.85714285714286
batch: 846 , accuracy: 89.52380952380953
batch: 847 , accuracy: 85.71428571428571
batch: 848 , accuracy: 83.80952380952381
batch: 849 , accuracy: 78.57142857142857
batch: 850 , accuracy: 78.0952380952381
batch: 851 , accu

batch: 1144 , accuracy: 76.66666666666667
batch: 1145 , accuracy: 78.57142857142857
batch: 1146 , accuracy: 84.28571428571429
batch: 1147 , accuracy: 85.71428571428571
batch: 1148 , accuracy: 91.42857142857143
batch: 1149 , accuracy: 92.85714285714286
batch: 1150 , accuracy: 92.85714285714286
batch: 1151 , accuracy: 92.85714285714286
batch: 1152 , accuracy: 92.85714285714286
batch: 1153 , accuracy: 92.85714285714286
batch: 1154 , accuracy: 92.85714285714286
batch: 1155 , accuracy: 92.85714285714286
batch: 1156 , accuracy: 92.85714285714286
batch: 1157 , accuracy: 92.85714285714286
batch: 1158 , accuracy: 92.85714285714286
batch: 1159 , accuracy: 92.85714285714286
batch: 1160 , accuracy: 92.85714285714286
batch: 1161 , accuracy: 92.85714285714286
batch: 1162 , accuracy: 92.85714285714286
batch: 1163 , accuracy: 92.85714285714286
batch: 1164 , accuracy: 92.85714285714286
batch: 1165 , accuracy: 92.85714285714286
batch: 1166 , accuracy: 86.66666666666667
batch: 1167 , accuracy: 85.7142857

...

batch: 4403 , accuracy: 92.85714285714286
batch: 4404 , accuracy: 92.85714285714286
batch: 4405 , accuracy: 92.85714285714286
batch: 4406 , accuracy: 92.85714285714286
batch: 4407 , accuracy: 92.85714285714286
batch: 4408 , accuracy: 92.85714285714286
batch: 4409 , accuracy: 92.85714285714286
batch: 4410 , accuracy: 92.85714285714286
batch: 4411 , accuracy: 95.71428571428572
batch: 4412 , accuracy: 100.0
batch: 4413 , accuracy: 100.0
batch: 4414 , accuracy: 100.0
batch: 4415 , accuracy: 100.0
batch: 4416 , accuracy: 100.0
batch: 4417 , accuracy: 97.14285714285714
batch: 4418 , accuracy: 92.85714285714286
batch: 4419 , accuracy: 91.42857142857143
batch: 4420 , accuracy: 85.71428571428571
batch: 4421 , accuracy: 85.71428571428571
batch: 4422 , accuracy: 78.57142857142857
batch: 4423 , accuracy: 78.57142857142857
batch: 4424 , accuracy: 72.85714285714285
batch: 4425 , accuracy: 71.42857142857143
batch: 4426 , accuracy: 67.14285714285714
batch: 4427 , accuracy: 64.28571428571429
batch: 442

batch: 4741 , accuracy: 82.85714285714286
batch: 4742 , accuracy: 78.57142857142857
batch: 4743 , accuracy: 70.47619047619048
batch: 4744 , accuracy: 64.28571428571429
batch: 4745 , accuracy: 64.28571428571429
batch: 4746 , accuracy: 57.14285714285714
batch: 4747 , accuracy: 57.14285714285714
batch: 4748 , accuracy: 51.42857142857142
batch: 4749 , accuracy: 50.0
batch: 4750 , accuracy: 50.0
batch: 4751 , accuracy: 50.0
batch: 4752 , accuracy: 54.285714285714285
batch: 4753 , accuracy: 57.14285714285714
batch: 4754 , accuracy: 57.14285714285714
batch: 4755 , accuracy: 57.14285714285714
batch: 4756 , accuracy: 57.14285714285714
batch: 4757 , accuracy: 57.14285714285714
batch: 4758 , accuracy: 57.14285714285714
batch: 4759 , accuracy: 57.14285714285714
batch: 4760 , accuracy: 57.14285714285714
batch: 4761 , accuracy: 57.14285714285714
batch: 4762 , accuracy: 60.0
batch: 4763 , accuracy: 64.28571428571429
batch: 4764 , accuracy: 67.61904761904762
batch: 4765 , accuracy: 71.42857142857143
b

batch: 5020 , accuracy: 57.14285714285714
batch: 5021 , accuracy: 57.14285714285714
batch: 5022 , accuracy: 57.14285714285714
batch: 5023 , accuracy: 57.14285714285714
batch: 5024 , accuracy: 57.14285714285714
batch: 5025 , accuracy: 57.14285714285714
batch: 5026 , accuracy: 57.14285714285714
batch: 5027 , accuracy: 57.14285714285714
batch: 5028 , accuracy: 57.61904761904761
batch: 5029 , accuracy: 64.28571428571429
batch: 5030 , accuracy: 64.76190476190476
batch: 5031 , accuracy: 71.42857142857143
batch: 5032 , accuracy: 72.38095238095238
batch: 5033 , accuracy: 78.57142857142857
batch: 5034 , accuracy: 79.04761904761905
batch: 5035 , accuracy: 78.57142857142857
batch: 5036 , accuracy: 80.0
batch: 5037 , accuracy: 85.71428571428571
batch: 5038 , accuracy: 87.61904761904762
batch: 5039 , accuracy: 92.85714285714286
batch: 5040 , accuracy: 92.85714285714286
batch: 5041 , accuracy: 92.85714285714286
batch: 5042 , accuracy: 92.85714285714286
batch: 5043 , accuracy: 96.19047619047619
batch

batch: 5318 , accuracy: 100.0
batch: 5319 , accuracy: 100.0
batch: 5320 , accuracy: 100.0
batch: 5321 , accuracy: 100.0
batch: 5322 , accuracy: 100.0
batch: 5323 , accuracy: 93.80952380952381
batch: 5324 , accuracy: 92.85714285714286
batch: 5325 , accuracy: 88.09523809523809
batch: 5326 , accuracy: 85.71428571428571
batch: 5327 , accuracy: 82.38095238095238
batch: 5328 , accuracy: 78.57142857142857
batch: 5329 , accuracy: 76.66666666666667
batch: 5330 , accuracy: 68.57142857142857
batch: 5331 , accuracy: 63.8095238095238
batch: 5332 , accuracy: 57.14285714285714
batch: 5333 , accuracy: 57.14285714285714
batch: 5334 , accuracy: 50.95238095238095
batch: 5335 , accuracy: 50.0
batch: 5336 , accuracy: 50.0
batch: 5337 , accuracy: 50.0
batch: 5338 , accuracy: 50.0
batch: 5339 , accuracy: 50.0
batch: 5340 , accuracy: 50.0
batch: 5341 , accuracy: 50.0
batch: 5342 , accuracy: 50.0
batch: 5343 , accuracy: 54.285714285714285
batch: 5344 , accuracy: 57.14285714285714
batch: 5345 , accuracy: 57.142

batch: 5592 , accuracy: 81.9047619047619
batch: 5593 , accuracy: 84.76190476190476
batch: 5594 , accuracy: 74.28571428571429
batch: 5595 , accuracy: 71.42857142857143
batch: 5596 , accuracy: 64.76190476190476
batch: 5597 , accuracy: 64.28571428571429
batch: 5598 , accuracy: 59.04761904761905
batch: 5599 , accuracy: 57.14285714285714
batch: 5600 , accuracy: 53.333333333333336
batch: 5601 , accuracy: 50.0
batch: 5602 , accuracy: 50.0
batch: 5603 , accuracy: 50.0
batch: 5604 , accuracy: 50.0
batch: 5605 , accuracy: 50.0
batch: 5606 , accuracy: 50.0
batch: 5607 , accuracy: 50.0
batch: 5608 , accuracy: 50.0
batch: 5609 , accuracy: 50.0
batch: 5610 , accuracy: 50.0
batch: 5611 , accuracy: 50.0
batch: 5612 , accuracy: 50.0
batch: 5613 , accuracy: 50.0
batch: 5614 , accuracy: 50.95238095238095
batch: 5615 , accuracy: 57.14285714285714
batch: 5616 , accuracy: 58.0952380952381
batch: 5617 , accuracy: 64.28571428571429
batch: 5618 , accuracy: 65.71428571428571
batch: 5619 , accuracy: 71.428571428

In [94]:
with tf.Session() as sess:
    # sess.run(tf.initialize_all_variables())
    sess.run(tf.global_variables_initializer())
    # plt.ion()
    # plt.figure()
    # plt.show()
    loss_list = []
    
    print("TRAINING")
    for epoch_idx in range(10): # 100
        _current_state = np.zeros((batch_size, state_size))

        print("New data, epoch", epoch_idx)

        for batch_idx in range(num_batches): # 666
            start_idx = batch_idx * truncated_backprop_length
            end_idx = start_idx + truncated_backprop_length
            # all 5 rows, certain numbers of column which the total = 15 columns
            batchX = x[:,start_idx:end_idx]
            batchY = y[:,start_idx:end_idx]
            
            # calculate loss, gradient step, 
            _total_loss, _train_step, _current_state, _predictions_series, _labels_series = sess.run(
                [total_loss, train_step, current_state, predictions_series, labels_series],
                feed_dict={
                    batchX_placeholder:batchX,
                    batchY_placeholder:batchY,
                    # it was part of computational graph, where the init_state is the placeholder
                    init_state:_current_state
                })

            loss_list.append(_total_loss)
            # print(_predictions_series)
            # print(_labels_series)
           
            if (batch_idx%(num_batches//10) == 0):
                print("Step",batch_idx, "Loss", _total_loss)
                # plot(loss_list, _predictions_series, batchX, batchY)

    # accuracy
    print("TESTING")
    _current_state = np.zeros((batch_size, state_size))
    acc = 0
    total_acc = 0
    for batch_idx in range(num_batches):
        start_idx = batch_idx * truncated_backprop_length
        end_idx = start_idx + truncated_backprop_length
        
        batchX = x[:,start_idx:end_idx]
        batchY = y[:,start_idx:end_idx]

        # calculate loss, gradient step, 
        _predictions_series, _labels_series = sess.run([predictions_series, labels_series],
            feed_dict={
                batchX_placeholder:batchX,
                batchY_placeholder:batchY,
                # it was part of computational graph, where the init_state is the placeholder
                init_state:_current_state
            })
        acc = accuracy_independent( _predictions_series, _labels_series)
        print("batch:",batch_idx,", accuracy:",acc)
        total_acc += acc
    
    print(total_acc/num_batches)
# plt.ioff()
# plt.show()

TRAINING
New data, epoch 0
Step 0 Loss 10228.295
Step 564 Loss 0.37885645
Step 1128 Loss 0.4072244
Step 1692 Loss 0.35937876
Step 2256 Loss 0.67396027
Step 2820 Loss 0.4973421
Step 3384 Loss 0.6704417
Step 3948 Loss 0.5029925
Step 4512 Loss 0.2699592
Step 5076 Loss 0.6891597
Step 5640 Loss 0.40415952
New data, epoch 1
Step 0 Loss 0.4953727
Step 564 Loss 0.36024293
Step 1128 Loss 0.41617104
Step 1692 Loss 0.36205372
Step 2256 Loss 0.67391807
Step 2820 Loss 0.50996786
Step 3384 Loss 0.6754765
Step 3948 Loss 0.49627653
Step 4512 Loss 0.27171203
Step 5076 Loss 0.6877245
Step 5640 Loss 0.40826637
New data, epoch 2
Step 0 Loss 0.4821383
Step 564 Loss 0.35221732
Step 1128 Loss 0.42617032
Step 1692 Loss 0.36173293
Step 2256 Loss 0.67336863
Step 2820 Loss 0.5153451
Step 3384 Loss 0.6793434
Step 3948 Loss 0.4912381
Step 4512 Loss 0.27349538
Step 5076 Loss 0.6858192
Step 5640 Loss 0.41141292
New data, epoch 3
Step 0 Loss 0.47484848
Step 564 Loss 0.3474071
Step 1128 Loss 0.43432796
Step 1692 Loss 

batch: 234 , accuracy: 59.04761904761905
batch: 235 , accuracy: 64.28571428571429
batch: 236 , accuracy: 66.19047619047619
batch: 237 , accuracy: 71.42857142857143
batch: 238 , accuracy: 73.80952380952381
batch: 239 , accuracy: 78.57142857142857
batch: 240 , accuracy: 80.95238095238095
batch: 241 , accuracy: 85.71428571428571
batch: 242 , accuracy: 88.57142857142857
batch: 243 , accuracy: 92.85714285714286
batch: 244 , accuracy: 92.85714285714286
batch: 245 , accuracy: 92.85714285714286
batch: 246 , accuracy: 92.85714285714286
batch: 247 , accuracy: 92.85714285714286
batch: 248 , accuracy: 92.85714285714286
batch: 249 , accuracy: 92.85714285714286
batch: 250 , accuracy: 92.85714285714286
batch: 251 , accuracy: 92.85714285714286
batch: 252 , accuracy: 92.85714285714286
batch: 253 , accuracy: 92.85714285714286
batch: 254 , accuracy: 92.85714285714286
batch: 255 , accuracy: 92.85714285714286
batch: 256 , accuracy: 92.85714285714286
batch: 257 , accuracy: 92.85714285714286
batch: 258 , acc

batch: 564 , accuracy: 92.85714285714286
batch: 565 , accuracy: 92.85714285714286
batch: 566 , accuracy: 92.85714285714286
batch: 567 , accuracy: 92.85714285714286
batch: 568 , accuracy: 92.85714285714286
batch: 569 , accuracy: 92.85714285714286
batch: 570 , accuracy: 92.85714285714286
batch: 571 , accuracy: 92.85714285714286
batch: 572 , accuracy: 92.85714285714286
batch: 573 , accuracy: 92.85714285714286
batch: 574 , accuracy: 92.85714285714286
batch: 575 , accuracy: 92.85714285714286
batch: 576 , accuracy: 92.85714285714286
batch: 577 , accuracy: 92.85714285714286
batch: 578 , accuracy: 92.85714285714286
batch: 579 , accuracy: 92.85714285714286
batch: 580 , accuracy: 87.14285714285714
batch: 581 , accuracy: 85.71428571428571
batch: 582 , accuracy: 81.42857142857143
batch: 583 , accuracy: 78.57142857142857
batch: 584 , accuracy: 75.71428571428571
batch: 585 , accuracy: 71.42857142857143
batch: 586 , accuracy: 70.0
batch: 587 , accuracy: 64.28571428571429
batch: 588 , accuracy: 64.285

batch: 844 , accuracy: 92.85714285714286
batch: 845 , accuracy: 92.85714285714286
batch: 846 , accuracy: 89.52380952380953
batch: 847 , accuracy: 85.71428571428571
batch: 848 , accuracy: 83.80952380952381
batch: 849 , accuracy: 78.57142857142857
batch: 850 , accuracy: 78.0952380952381
batch: 851 , accuracy: 71.42857142857143
batch: 852 , accuracy: 71.42857142857143
batch: 853 , accuracy: 65.23809523809524
batch: 854 , accuracy: 64.28571428571429
batch: 855 , accuracy: 59.523809523809526
batch: 856 , accuracy: 57.14285714285714
batch: 857 , accuracy: 53.80952380952381
batch: 858 , accuracy: 50.0
batch: 859 , accuracy: 50.0
batch: 860 , accuracy: 50.0
batch: 861 , accuracy: 50.0
batch: 862 , accuracy: 50.0
batch: 863 , accuracy: 50.0
batch: 864 , accuracy: 50.0
batch: 865 , accuracy: 50.0
batch: 866 , accuracy: 50.0
batch: 867 , accuracy: 50.0
batch: 868 , accuracy: 50.0
batch: 869 , accuracy: 50.0
batch: 870 , accuracy: 50.0
batch: 871 , accuracy: 50.0
batch: 872 , accuracy: 57.14285714

batch: 1129 , accuracy: 50.0
batch: 1130 , accuracy: 50.0
batch: 1131 , accuracy: 50.0
batch: 1132 , accuracy: 50.0
batch: 1133 , accuracy: 50.0
batch: 1134 , accuracy: 50.0
batch: 1135 , accuracy: 50.0
batch: 1136 , accuracy: 50.0
batch: 1137 , accuracy: 50.0
batch: 1138 , accuracy: 54.761904761904766
batch: 1139 , accuracy: 57.14285714285714
batch: 1140 , accuracy: 61.904761904761905
batch: 1141 , accuracy: 64.28571428571429
batch: 1142 , accuracy: 69.52380952380952
batch: 1143 , accuracy: 71.42857142857143
batch: 1144 , accuracy: 76.66666666666667
batch: 1145 , accuracy: 78.57142857142857
batch: 1146 , accuracy: 84.28571428571429
batch: 1147 , accuracy: 85.71428571428571
batch: 1148 , accuracy: 91.42857142857143
batch: 1149 , accuracy: 92.85714285714286
batch: 1150 , accuracy: 92.85714285714286
batch: 1151 , accuracy: 92.85714285714286
batch: 1152 , accuracy: 92.85714285714286
batch: 1153 , accuracy: 92.85714285714286
batch: 1154 , accuracy: 92.85714285714286
batch: 1155 , accuracy:

...

batch: 4413 , accuracy: 100.0
batch: 4414 , accuracy: 100.0
batch: 4415 , accuracy: 100.0
batch: 4416 , accuracy: 100.0
batch: 4417 , accuracy: 97.14285714285714
batch: 4418 , accuracy: 92.85714285714286
batch: 4419 , accuracy: 91.42857142857143
batch: 4420 , accuracy: 85.71428571428571
batch: 4421 , accuracy: 85.71428571428571
batch: 4422 , accuracy: 78.57142857142857
batch: 4423 , accuracy: 78.57142857142857
batch: 4424 , accuracy: 72.85714285714285
batch: 4425 , accuracy: 71.42857142857143
batch: 4426 , accuracy: 67.14285714285714
batch: 4427 , accuracy: 64.28571428571429
batch: 4428 , accuracy: 64.28571428571429
batch: 4429 , accuracy: 64.28571428571429
batch: 4430 , accuracy: 59.523809523809526
batch: 4431 , accuracy: 57.14285714285714
batch: 4432 , accuracy: 57.14285714285714
batch: 4433 , accuracy: 57.14285714285714
batch: 4434 , accuracy: 57.14285714285714
batch: 4435 , accuracy: 57.14285714285714
batch: 4436 , accuracy: 57.14285714285714
batch: 4437 , accuracy: 57.142857142857

batch: 4717 , accuracy: 91.42857142857143
batch: 4718 , accuracy: 92.85714285714286
batch: 4719 , accuracy: 99.04761904761905
batch: 4720 , accuracy: 100.0
batch: 4721 , accuracy: 100.0
batch: 4722 , accuracy: 100.0
batch: 4723 , accuracy: 100.0
batch: 4724 , accuracy: 100.0
batch: 4725 , accuracy: 100.0
batch: 4726 , accuracy: 100.0
batch: 4727 , accuracy: 100.0
batch: 4728 , accuracy: 100.0
batch: 4729 , accuracy: 100.0
batch: 4730 , accuracy: 100.0
batch: 4731 , accuracy: 100.0
batch: 4732 , accuracy: 100.0
batch: 4733 , accuracy: 100.0
batch: 4734 , accuracy: 100.0
batch: 4735 , accuracy: 100.0
batch: 4736 , accuracy: 100.0
batch: 4737 , accuracy: 94.28571428571428
batch: 4738 , accuracy: 92.85714285714286
batch: 4739 , accuracy: 88.57142857142857
batch: 4740 , accuracy: 85.71428571428571
batch: 4741 , accuracy: 82.85714285714286
batch: 4742 , accuracy: 78.57142857142857
batch: 4743 , accuracy: 70.47619047619048
batch: 4744 , accuracy: 64.28571428571429
batch: 4745 , accuracy: 64.2

batch: 4986 , accuracy: 100.0
batch: 4987 , accuracy: 100.0
batch: 4988 , accuracy: 100.0
batch: 4989 , accuracy: 100.0
batch: 4990 , accuracy: 100.0
batch: 4991 , accuracy: 100.0
batch: 4992 , accuracy: 100.0
batch: 4993 , accuracy: 100.0
batch: 4994 , accuracy: 100.0
batch: 4995 , accuracy: 100.0
batch: 4996 , accuracy: 100.0
batch: 4997 , accuracy: 100.0
batch: 4998 , accuracy: 100.0
batch: 4999 , accuracy: 96.19047619047619
batch: 5000 , accuracy: 92.85714285714286
batch: 5001 , accuracy: 92.85714285714286
batch: 5002 , accuracy: 92.85714285714286
batch: 5003 , accuracy: 89.52380952380953
batch: 5004 , accuracy: 85.71428571428571
batch: 5005 , accuracy: 83.80952380952381
batch: 5006 , accuracy: 78.57142857142857
batch: 5007 , accuracy: 80.0
batch: 5008 , accuracy: 78.57142857142857
batch: 5009 , accuracy: 78.57142857142857
batch: 5010 , accuracy: 72.38095238095238
batch: 5011 , accuracy: 71.42857142857143
batch: 5012 , accuracy: 66.66666666666666
batch: 5013 , accuracy: 64.28571428

batch: 5311 , accuracy: 94.76190476190476
batch: 5312 , accuracy: 100.0
batch: 5313 , accuracy: 100.0
batch: 5314 , accuracy: 100.0
batch: 5315 , accuracy: 100.0
batch: 5316 , accuracy: 100.0
batch: 5317 , accuracy: 100.0
batch: 5318 , accuracy: 100.0
batch: 5319 , accuracy: 100.0
batch: 5320 , accuracy: 100.0
batch: 5321 , accuracy: 100.0
batch: 5322 , accuracy: 100.0
batch: 5323 , accuracy: 93.80952380952381
batch: 5324 , accuracy: 92.85714285714286
batch: 5325 , accuracy: 88.09523809523809
batch: 5326 , accuracy: 85.71428571428571
batch: 5327 , accuracy: 82.38095238095238
batch: 5328 , accuracy: 78.57142857142857
batch: 5329 , accuracy: 76.66666666666667
batch: 5330 , accuracy: 68.57142857142857
batch: 5331 , accuracy: 63.8095238095238
batch: 5332 , accuracy: 57.14285714285714
batch: 5333 , accuracy: 57.14285714285714
batch: 5334 , accuracy: 50.95238095238095
batch: 5335 , accuracy: 50.0
batch: 5336 , accuracy: 50.0
batch: 5337 , accuracy: 50.0
batch: 5338 , accuracy: 50.0
batch: 53

batch: 5554 , accuracy: 50.0
batch: 5555 , accuracy: 50.0
batch: 5556 , accuracy: 50.0
batch: 5557 , accuracy: 50.0
batch: 5558 , accuracy: 50.0
batch: 5559 , accuracy: 50.0
batch: 5560 , accuracy: 50.0
batch: 5561 , accuracy: 52.85714285714286
batch: 5562 , accuracy: 64.28571428571429
batch: 5563 , accuracy: 67.14285714285714
batch: 5564 , accuracy: 71.42857142857143
batch: 5565 , accuracy: 72.85714285714285
batch: 5566 , accuracy: 71.42857142857143
batch: 5567 , accuracy: 74.76190476190476
batch: 5568 , accuracy: 78.57142857142857
batch: 5569 , accuracy: 82.38095238095238
batch: 5570 , accuracy: 85.71428571428571
batch: 5571 , accuracy: 89.52380952380953
batch: 5572 , accuracy: 92.85714285714286
batch: 5573 , accuracy: 92.85714285714286
batch: 5574 , accuracy: 92.85714285714286
batch: 5575 , accuracy: 92.85714285714286
batch: 5576 , accuracy: 92.85714285714286
batch: 5577 , accuracy: 92.85714285714286
batch: 5578 , accuracy: 92.85714285714286
batch: 5579 , accuracy: 92.85714285714286

In [ ]:
#LTSM

In [ ]:
from __future__ import print_function, division
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [63]:
num_epochs = 100
total_series_length = num_samples
truncated_backprop_length = 15
state_size = 1
num_classes = 2
echo_step = 3
batch_size = 7
num_batches = total_series_length//batch_size//truncated_backprop_length

In [64]:
batch_size
state_size
num_batches

5648

In [62]:
truncated_backprop_length

15

In [59]:
num_epochs
num_samples
num_classes

2

In [ ]:
def generateData():
    x = np.array(np.random.choice(2, total_series_length, p=[0.5, 0.5]))
    y = np.roll(x, echo_step)
    y[0:echo_step] = 0

    x = x.reshape((batch_size, -1))  # The first index changing slowest, subseries as rows
    y = y.reshape((batch_size, -1))

    return (x, y)

In [65]:
batchX_placeholder = tf.placeholder(tf.float32, [batch_size, truncated_backprop_length])
batchY_placeholder = tf.placeholder(tf.float32, [batch_size, truncated_backprop_length, num_classes])

# init_state = tf.placeholder(tf.float32, [batch_size, state_size])
cell_state = tf.placeholder(tf.float32, [batch_size, state_size])
hidden_state = tf.placeholder(tf.float32, [batch_size, state_size])
init_state = tf.nn.rnn_cell.LSTMStateTuple(cell_state, hidden_state)

W2 = tf.Variable(np.random.rand(state_size, num_classes),dtype=tf.float32)
b2 = tf.Variable(np.zeros((1,num_classes)), dtype=tf.float32)

In [68]:
# Unpack columns
inputs_series = tf.split(batchX_placeholder, truncated_backprop_length, 1)
labels_series = tf.unstack(batchY_placeholder, axis=1)

In [69]:
inputs_series

[<tf.Tensor 'split_2:0' shape=(7, 1) dtype=float32>,
 <tf.Tensor 'split_2:1' shape=(7, 1) dtype=float32>,
 <tf.Tensor 'split_2:2' shape=(7, 1) dtype=float32>,
 <tf.Tensor 'split_2:3' shape=(7, 1) dtype=float32>,
 <tf.Tensor 'split_2:4' shape=(7, 1) dtype=float32>,
 <tf.Tensor 'split_2:5' shape=(7, 1) dtype=float32>,
 <tf.Tensor 'split_2:6' shape=(7, 1) dtype=float32>,
 <tf.Tensor 'split_2:7' shape=(7, 1) dtype=float32>,
 <tf.Tensor 'split_2:8' shape=(7, 1) dtype=float32>,
 <tf.Tensor 'split_2:9' shape=(7, 1) dtype=float32>,
 <tf.Tensor 'split_2:10' shape=(7, 1) dtype=float32>,
 <tf.Tensor 'split_2:11' shape=(7, 1) dtype=float32>,
 <tf.Tensor 'split_2:12' shape=(7, 1) dtype=float32>,
 <tf.Tensor 'split_2:13' shape=(7, 1) dtype=float32>,
 <tf.Tensor 'split_2:14' shape=(7, 1) dtype=float32>]

In [70]:
labels_series

[<tf.Tensor 'unstack_2:0' shape=(7, 2) dtype=float32>,
 <tf.Tensor 'unstack_2:1' shape=(7, 2) dtype=float32>,
 <tf.Tensor 'unstack_2:2' shape=(7, 2) dtype=float32>,
 <tf.Tensor 'unstack_2:3' shape=(7, 2) dtype=float32>,
 <tf.Tensor 'unstack_2:4' shape=(7, 2) dtype=float32>,
 <tf.Tensor 'unstack_2:5' shape=(7, 2) dtype=float32>,
 <tf.Tensor 'unstack_2:6' shape=(7, 2) dtype=float32>,
 <tf.Tensor 'unstack_2:7' shape=(7, 2) dtype=float32>,
 <tf.Tensor 'unstack_2:8' shape=(7, 2) dtype=float32>,
 <tf.Tensor 'unstack_2:9' shape=(7, 2) dtype=float32>,
 <tf.Tensor 'unstack_2:10' shape=(7, 2) dtype=float32>,
 <tf.Tensor 'unstack_2:11' shape=(7, 2) dtype=float32>,
 <tf.Tensor 'unstack_2:12' shape=(7, 2) dtype=float32>,
 <tf.Tensor 'unstack_2:13' shape=(7, 2) dtype=float32>,
 <tf.Tensor 'unstack_2:14' shape=(7, 2) dtype=float32>]

In [72]:
from tensorflow.contrib import rnn 

In [85]:
# Forward passes
# cell = tf.nn.rnn_cell.BasicRNNCell(state_size)
# states_series, current_state = tf.nn.rnn(cell, inputs_series, init_state)
# cell = tf.nn.rnn_cell.BasicLSTMCell(state_size, state_is_tuple=True)
cell = rnn.BasicLSTMCell(state_size, reuse=True)
# states_series, current_state = tf.nn.rnn(cell, inputs_series, init_state)
states_series, current_state = rnn.static_rnn(cell, inputs_series, init_state)

logits_series = [tf.matmul(state, W2) + b2 for state in states_series] #Broadcasted addition
# predictions_series = [tf.nn.softmax(logits) for logits in logits_series]
predictions_series = [tf.round(tf.sigmoid(logits)) for logits in logits_series]

losses = [tf.nn.sigmoid_cross_entropy_with_logits(logits = logits,labels = labels) for logits, labels in zip(logits_series,labels_series)]
total_loss = tf.reduce_mean(losses)

train_step = tf.train.AdagradOptimizer(0.3).minimize(total_loss)

ValueError: Variable rnn/basic_lstm_cell/kernel/Adagrad/ already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "c:\users\gebruiker\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\ops.py", line 1650, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "c:\users\gebruiker\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\ops.py", line 3271, in create_op
    op_def=op_def)
  File "c:\users\gebruiker\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 787, in _apply_op_helper
    op_def=op_def)


In [ ]:
def plot(loss_list, predictions_series, batchX, batchY):
    plt.subplot(2, 3, 1)
    plt.cla()
    plt.plot(loss_list)

    for batch_series_idx in range(5):
        one_hot_output_series = np.array(predictions_series)[:, batch_series_idx, :]
        single_output_series = np.array([(1 if out[0] < 0.5 else 0) for out in one_hot_output_series])

        plt.subplot(2, 3, batch_series_idx + 2)
        plt.cla()
        plt.axis([0, truncated_backprop_length, 0, 2])
        left_offset = range(truncated_backprop_length)
        plt.bar(left_offset, batchX[batch_series_idx, :], width=1, color="blue")
        plt.bar(left_offset, batchY[batch_series_idx, :] * 0.5, width=1, color="red")
        plt.bar(left_offset, single_output_series * 0.3, width=1, color="green")

    # plt.draw()
    # plt.pause(0.0001)

In [77]:
x.shape

(7, 84727)

In [78]:
y.shape

(7, 84727, 2)

In [82]:
with tf.Session() as sess:
    # sess.run(tf.initialize_all_variables())
    sess.run(tf.global_variables_initializer())
    # plt.ion()
    # plt.figure()
    # plt.show()
    loss_list = []

    for epoch_idx in range(1):
        # x,y = generateData()
        # _current_state = np.zeros((batch_size, state_size))
        _current_cell_state = np.zeros((batch_size, state_size))
        _current_hidden_state = np.zeros((batch_size, state_size))

        print("New data, epoch", epoch_idx)

        for batch_idx in range(num_batches):
            start_idx = batch_idx * truncated_backprop_length
            end_idx = start_idx + truncated_backprop_length

            batchX = x[:,start_idx:end_idx]
            batchY = y[:,start_idx:end_idx]

            """
            _total_loss, _train_step, _current_state, _predictions_series = sess.run(
                [total_loss, train_step, current_state, predictions_series],
                feed_dict={
                    batchX_placeholder:batchX,
                    batchY_placeholder:batchY,
                    init_state:_current_state
                })
            """
            
            _total_loss, _train_step, _current_state, _predictions_series, _labels_series = sess.run(
                [total_loss, train_step, current_state, predictions_series, labels_series],
                feed_dict={
                    batchX_placeholder: batchX,
                    batchY_placeholder: batchY,
                    cell_state: _current_cell_state,
                    hidden_state: _current_hidden_state
                })
            
            _current_cell_state, _current_hidden_state = _current_state
            
            loss_list.append(_total_loss)
            # print(_predictions_series)
            # print(_labels_series)
            
            if batch_idx%100 == 0:
                print("Step",batch_idx, "Loss", _total_loss)
                # plot(loss_list, _predictions_series, batchX, batchY)
    # accuracy
    print("TESTING")
    # _current_state = np.zeros((batch_size, state_size))
    _current_cell_state = np.zeros((batch_size, state_size))
    _current_hidden_state = np.zeros((batch_size, state_size))
    acc = 0
    total_acc = 0
    for batch_idx in range(num_batches):
        start_idx = batch_idx * truncated_backprop_length
        end_idx = start_idx + truncated_backprop_length
        
        batchX = x[:,start_idx:end_idx]
        batchY = y[:,start_idx:end_idx]
        
        _current_state, _predictions_series, _labels_series = sess.run(
                [current_state, predictions_series, labels_series],
                feed_dict={
                    batchX_placeholder: batchX,
                    batchY_placeholder: batchY,
                    cell_state: _current_cell_state,
                    hidden_state: _current_hidden_state
                })
            
        _current_cell_state, _current_hidden_state = _current_state
            
        acc = accuracy_independent( _predictions_series, _labels_series)
        print("batch:",batch_idx,", accuracy:",acc)
        total_acc += acc
    
    print(total_acc/num_batches)
# plt.ioff()
# plt.show()


New data, epoch 0
[array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]], dtype=float32), array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]], dtype=float32), array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]], dtype=float32), array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]], dtype=float32), array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]], dtype=float32), array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]], dtype=float32), array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]], dtype=float32), array([[0., 0.],
       [0., 

[array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1

       [0., 1.]], dtype=float32)]
[array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32), array([[0., 1

[array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1

[array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1

...

[array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0., 1.],
       [0., 1.],
       [0., 1

[array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32), array([[1., 1.],
       [1., 1.],
       [1., 1

[array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32), array([[1., 1.],
       [1., 1.],
       [1., 1

[array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32), array([[1., 1.],
       [1., 1.],
       [1., 1

KeyboardInterrupt: 

In [ ]:
def plot(loss_list, predictions_series, batchX, batchY):
    plt.subplot(2, 3, 1)
    plt.cla()
    plt.plot(loss_list)

    for batch_series_idx in range(5):
        one_hot_output_series = np.array(predictions_series)[:, batch_series_idx, :]
        single_output_series = np.array([(1 if out[0] < 0.5 else 0) for out in one_hot_output_series])

        plt.subplot(2, 3, batch_series_idx + 2)
        plt.cla()
        plt.axis([0, truncated_backprop_length, 0, 2])
        left_offset = range(truncated_backprop_length)
        plt.bar(left_offset, batchX[batch_series_idx, :], width=1, color="blue")
        plt.bar(left_offset, batchY[batch_series_idx, :] * 0.5, width=1, color="red")
        plt.bar(left_offset, single_output_series * 0.3, width=1, color="green")

    plt.draw()
    plt.pause(0.0001)